In [2]:
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import contractions
import re
from nltk.tokenize import word_tokenize
import pandas as pd
import snscrape.modules.twitter as sntwitter


In [3]:
import nltk
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
def get_tweets(username):
    tweets = []
    for i, tweet in enumerate(sntwitter.TwitterSearchScraper(f'from:{username}').get_items()):
        tweets.append(tweet.content)
        if i == 50: break
    return tweets

In [5]:
def load_files():
    try:
        with open("../server/models/RandomForest_E-I.sav", "rb") as file:
            ei_classifier = pickle.load(file)
        with  open("../server/models/RandomForest_N-S.sav", "rb") as file:
            ns_classifier = pickle.load(file)
        with open("../server/models/RandomForest_F-T.sav", "rb") as file:
            ft_classifier = pickle.load(file)
        with  open("../server/models/RandomForest_J-P.sav", "rb") as file:
            jp_classifier = pickle.load(file)
    except FileNotFoundError:
        print("Model not found!")

    try:
        with open("../server/models/vectorizer.pkl", "rb") as file:
            vectorizer = pickle.load(file)
    except FileNotFoundError:
        print("Tokenizer not found!")

    return ei_classifier, ns_classifier, ft_classifier, jp_classifier, vectorizer
    



In [6]:
def preprocessing(text):
    stopword_list = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    text = contractions.fix(text)
    text = text.lower()
    text = re.sub(r'@([a-zA-Z0-9_]{1,50})', '', text)
    text = re.sub(r'#([a-zA-Z0-9_]{1,50})', '', text)
    text = re.sub(r'http[s]?://\S+', '', text)
    text = re.sub(r'[^A-Za-z0-9]+', ' ', text)
    text = re.sub(r' +', ' ', text)
    text = " ".join([word for word in text.split() if not len(word) <3])
    text = word_tokenize(text)
    text = [word for word in text if not word in stopword_list]
    text = [lemmatizer.lemmatize(word) for word in text]
    text = " ".join(text)
    return text

In [7]:
def get_prediction(username,tweet=[]):
    ei_classifier, ns_classifier, ft_classifier, jp_classifier, vectorizer = load_files()
    if(len(tweet)==0):
        tweets = get_tweets(username)
    else:
        tweets=tweet
      
    text   = " ".join(tweets)
    text   = preprocessing(text)
    text   = vectorizer.transform([text])
    
    prediction = ""
    e_or_i = "E" if ei_classifier.predict(text)[0] == 1 else "I"
    n_or_s = "N" if ns_classifier.predict(text)[0] == 1 else "S"
    f_or_t = "F" if ft_classifier.predict(text)[0] == 1 else "T"
    j_or_p = "J" if jp_classifier.predict(text)[0] == 1 else "P"
    prediction = e_or_i + n_or_s + f_or_t + j_or_p

    return prediction, tweets

In [8]:
get_prediction("billgates")

C:\Users\HP\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\HP\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\HP\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator SVC from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\HP\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at y

('INFJ',
 ["I was shocked to wake up to the news this morning. A reversal of Roe v. Wade would set us back 50 years and disproportionately impact the most vulnerable women in society.\n\nI support a woman's right to make their own decisions about their health care.",
  'My new book comes out tomorrow – but @GatesNotes Insiders can get an early sneak peek and download a free chapter now: https://t.co/6cQv2FwB4W https://t.co/QwbjHOnywy',
  'No one wants to live through this again – and we don’t have to. As @larrybrilliant says: Outbreaks are inevitable, but pandemics are optional. https://t.co/RSdJCbDyd3',
  'I’ve been lucky to spend time with, and learn from, many extraordinary teachers over the years, like these incredible Washington State teachers. I appreciate their dedication and resilience. #TeacherAppreciationWeek https://t.co/aZeLqEKFkz',
  'The world can make COVID-19 the last pandemic: https://t.co/b3bHaQ3eLr https://t.co/wnKIfARZzh',
  'The world wasn’t ready for COVID, but we

In [9]:
import tweepy
import pandas as pd
import time
import os

class TwitterClient(object):

    def __init__(self):

        #API KEY
        self.ckey='9dWxuFf6svWFW9XUP63xHEYMf' 
        #API SECRET
        self.csecret='Zkn65SEWsdpHbiBnnHStlOFcJD67KBDJOvdWDYJlGQLzShXA7u'
        #ACCESS TOKEN
        self.atoken='1015294450006757376-vmsspbAxG1yWnDHL65e7XqwtNYIPWd'
        #ACCESS SECRET
        self.asecret='xm2QnNP5vgeUvU7waASVOh9UlYUwj3zbeurkctBkPfDmU'

        try:
          
            auth=tweepy.OAuthHandler(self.ckey, self.csecret)
            auth.set_access_token(self.atoken, self.asecret)
            self.api=tweepy.API(auth)
            
        except tweepy.TweepError as e:
            print(f"Error: Twitter Authentication Failed - \n{str(e)}") 

    def get_user_tweets(self, user):
        username = user
        count = 100

        try:
            # Create query method using parameters
            tweets = tweepy.Cursor(self.api.user_timeline,screen_name=username).items(count)
            
            # Pull information from tweets iterable object
            tweets_list = [[tweet.text] for tweet in tweets]

            # Create dataframe from tweets list
            tweets_df = pd.DataFrame(tweets_list)
            #print(tweets_df)
            return tweets_df
           
        except BaseException as e:
            print('failed on_status,',str(e))
            time.sleep(3)

    def get_user_followers(self,user):
        # to fetch 5 following of the user and their personality types
        username = user
        follower_ids = []
        nfollowers = 5
        #print ("Getting following...")
        users = tweepy.Cursor(self.api.get_friends,screen_name = username,count=nfollowers).items(nfollowers)
        tweets_list = []
        for user in users:
            # tweet_user = []
            #print ("Adding following...")
            try:
                follower_ids.append(user.screen_name)
                tweets = tweepy.Cursor(self.api.user_timeline,screen_name=user.screen_name).items(100)
                # print ([tweet.text for tweet in tweets])
                tweets_list.append([tweet.text for tweet in tweets])
            except tweepy.errors.TweepError as e:
                print (e)
                time.sleep(60)
        #print (follower_ids)
        tweet_df = pd.DataFrame({'follower':follower_ids})
        tweet_df["tweets"] = pd.Series(tweets_list)
        #print (follower_ids)
        #print(tweet_df)
        return tweet_df

def tweet_return(tweet_handle):
    twitter = TwitterClient()

    tweet_path = os.path.join("../server/Dataset/", "tweets_"+str(tweet_handle)+".csv")
    tweet_follower_path = os.path.join("../server/Dataset/","follower_"+str(tweet_handle)+".csv")

    #tweet_path = "/MyDrive/MyDrive/MajorFinal/datasets/twitter_data/"+"tweets_"+str(tweet_handle)+".csv"
    #tweet_fol_path = "/MyDrive/MyDrive/MajorFinal/datasets/twitter_data/"+"fol_"+str(tweet_handle)+".csv"
    twitter.get_user_tweets(str(tweet_handle)).to_csv(tweet_path)
    twitter.get_user_followers(str(tweet_handle)).to_csv(tweet_follower_path)

In [10]:
tweet_return("billgates")

In [12]:
import string
import re
import pickle
import tensorflow as tf
from scipy import stats
import pandas as pd
import numpy as np

maxlen = 1500
per_types = ['ENFJ','ENFP','ENTJ','ENTP','ESFJ','ESFP','ESTJ','ESTP','INFJ','INFP','INTJ','INTP','ISFJ','ISFP','ISTJ','ISTP']

def clean_text(text):
    regex = re.compile('[%s]' % re.escape('|'))
    text = regex.sub(" ", text)
    words = str(text).split()
    words = [i.lower() + " " for i in words]
    words = [i for i in words if not "http" in i]
    words = " ".join(words)
    words = words.translate(words.maketrans('', '', string.punctuation))
    return words

def predict_type(text):
    cleaned_ip = clean_text(text)
    #custom_test_ids = [tokenizer.encode(str(cleaned_ip))]
    #type_ind = np.argmax(new_model.predict(np.array(custom_test_ids)))
    return get_prediction("abc",[cleaned_ip])[0]

def predict_tweet(username):
    get_prediction(username)
    
    per_op = get_prediction(username)[0]

    op_json = {}
    op_json["name"] = str(username)
    op_json["type"] = str(per_op)
    info_df = pd.read_csv("../server/Dataset/MBTI.csv")

    #print(per_op)
    #print(info_df)
    op_json["traits"] = info_df[info_df["type"]==per_op]["traits"].values[0]
    op_json["career"] = info_df[info_df["type"]==per_op]["career"].values[0]
    op_json["people"] = info_df[info_df["type"]==per_op]["eminent personalities"].values[0]
    op_json["per_name"] = info_df[info_df["type"]==per_op]["name"].values[0]
    perfile = open("../server/static/results.js","w")
    perstr = "var personality_data="+str(op_json)+"\n"
    perfile.write(perstr)
    perfile.close()
    print(op_json)
    return op_json

def predict_follow(username):
    twitter = TwitterClient()
    follow_df = twitter.get_user_followers(str(username))
    follow_json = {}
    follow_ids = {}
    for i in range(5):
        list_j = follow_df.tweets.iloc[i]
        new_list = []
        for j in list_j:
            new_list.append(clean_text(j))
        print(get_prediction(follow_df.follower.iloc[i],new_list)[0])
    
        tweet_ind = get_prediction(follow_df.follower.iloc[i],new_list)[0]
        follow_json[str(i)] = tweet_ind
        follow_ids[str(i)] = follow_df.follower.iloc[i]
        perfile = open("../server/static/results2.js","w")
        perstr = "var follower_data="+str(follow_json)+"\n"
        folstr = "var follower_ids="+str(follow_ids)+"\n"
        perfile.write(perstr)
        perfile.write(folstr)
        perfile.close()
    return follow_json
#predict_tweet("simran")

In [13]:
predict_tweet("simran")

{'name': 'simran', 'type': 'INFP', 'traits': 'Creative, Independent, Adaptable, Inquisitive, Caring', 'career': 'Writer, Bureaucrat, Social Worker, Psychologist, Teacher', 'people': 'William Shakespeare, Vincent Van Gogh, Hellen Keller, John Mayer', 'per_name': 'Mediator'}


{'name': 'simran',
 'type': 'INFP',
 'traits': 'Creative, Independent, Adaptable, Inquisitive, Caring',
 'career': 'Writer, Bureaucrat, Social Worker, Psychologist, Teacher',
 'people': 'William Shakespeare, Vincent Van Gogh, Hellen Keller, John Mayer',
 'per_name': 'Mediator'}

In [ ]:
from flask import Flask, render_template, request, redirect, url_for,jsonify
from flask_cors import CORS


app = Flask(__name__, static_folder="static")
#CORS(app)


@app.route('/')
def home():
    return render_template('homepage.html')

@app.route('/predict', methods=['GET', 'POST'])
def testfn():
    # GET request
    if request.method == 'GET':
        message = {'greeting':'Hello from Flask!'}
        return jsonify(message)  # serialize and use JSON headers
    # POST request
    if request.method == 'POST':
        data = request.get_json()  # parse as JSON
        user_text = data["data"]
        user_type = predict_type(user_text)
        return jsonify({"type":str(user_type)}),200

@app.route('/tweet_pred', methods=['GET', 'POST'])
def tweet():
    # GET request
    if request.method == 'GET':
        return render_template('dashboard.html')
    # POST request
    if request.method == 'POST':
        data = request.get_json()  # parse as JSON
        user_handle = data["handle"]
        tweet_return(user_handle)
        user_type = predict_tweet(user_handle)
        #render_template('result.html')
        return jsonify(user_type),200

@app.route('/follow_pred', methods=['GET', 'POST'])
#endpoint to fetch user followers
def follow_tweet():
    # GET request
    if request.method == 'GET':
        message = {'greeting':'Hello from Flask!'}
        return jsonify(message)  # serialize and use JSON headers
    # POST request
    if request.method == 'POST':
        data = request.get_json()  # parse as JSON
        user_handle = data["handle"]
        # tweet_return(user_handle)
        user_type = predict_follow(user_handle)
        return jsonify({"type":str(user_type)}),200

if __name__ == '__main__' :
    app.config['TEMPLATES_AUTO_RELOAD'] = True
    app.config['SEND_FILE_MAX_AGE_DEFAULT'] = 0
    app.run(debug=False, port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/May/2022 22:21:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2022 22:21:24] "GET /static/assets/vendor/venobox/venobox.css HTTP/1.1" 304 -
127.0.0.1 - - [03/May/2022 22:21:24] "GET /static/assets/vendor/remixicon/remixicon.css HTTP/1.1" 304 -
127.0.0.1 - - [03/May/2022 22:21:24] "GET /static/assets/vendor/bootstrap/css/bootstrap.min.css HTTP/1.1" 304 -
127.0.0.1 - - [03/May/2022 22:21:24] "GET /static/assets/vendor/aos/aos.css HTTP/1.1" 304 -
127.0.0.1 - - [03/May/2022 22:21:24] "GET /static/assets/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [03/May/2022 22:21:24] "GET /static/assets/vendor/boxicons/css/boxicons.min.css HTTP/1.1" 304 -
127.0.0.1 - - [03/May/2022 22:21:24] "GET /static/assets/vendor/jquery/jquery.min.js HTTP/1.1" 304 -
127.0.0.1 - - [03/May/2022 22:21:24] "GET /static/assets/vendor/icofont/icofont.min.css HTTP/1.1" 304 -
127.0.0.1 - - [03/May/2022 22:21:24] "GET /static/assets/vendor/bootst

{'name': 'elonmusk', 'type': 'INTP', 'traits': 'Conceptual, Complex, Intellectual, Critical, Ingenious', 'career': 'Programmer, Computer Scientist, Web Developer, Composer, Economist, Scientist', 'people': 'Larry Page, Albert Einstein, Charles Darwin, Abraham Lincoln, ', 'per_name': 'Logician'}


127.0.0.1 - - [03/May/2022 22:23:07] "GET /tweet_pred HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2022 22:23:07] "GET /static/assets2/vendors/iconfonts/mdi/css/materialdesignicons.min.css HTTP/1.1" 304 -
127.0.0.1 - - [03/May/2022 22:23:07] "GET /static/assets2/vendors/iconfonts/ionicons/dist/css/ionicons.css HTTP/1.1" 304 -
127.0.0.1 - - [03/May/2022 22:23:07] "GET /static/assets2/vendors/iconfonts/flag-icon-css/css/flag-icon.min.css HTTP/1.1" 304 -
127.0.0.1 - - [03/May/2022 22:23:07] "GET /static/assets2/vendors/css/vendor.bundle.base.css HTTP/1.1" 304 -
127.0.0.1 - - [03/May/2022 22:23:07] "GET /static/assets2/vendors/css/vendor.bundle.addons.css HTTP/1.1" 304 -
127.0.0.1 - - [03/May/2022 22:23:07] "GET /static/assets2/css/shared/style.css HTTP/1.1" 304 -
127.0.0.1 - - [03/May/2022 22:23:07] "GET /static/assets2/css/demo_1/style.css HTTP/1.1" 304 -
127.0.0.1 - - [03/May/2022 22:23:07] "GET /static/results2.js HTTP/1.1" 304 -
127.0.0.1 - - [03/May/2022 22:23:07] "GET /static/results.js HT

{'name': '', 'type': 'INTP', 'traits': 'Conceptual, Complex, Intellectual, Critical, Ingenious', 'career': 'Programmer, Computer Scientist, Web Developer, Composer, Economist, Scientist', 'people': 'Larry Page, Albert Einstein, Charles Darwin, Abraham Lincoln, ', 'per_name': 'Logician'}
